In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

In [3]:
#Note 1/18/23: Flux tower footprint (fluxproper) OR expanded flux area (fluxdiss) OR all gce (gce_clip)

gce_clip = 'F:/Wetlands shapefiles/GA_shapefile_wetlands/GA_sapelodissolve.shp'

gce_fluxdiss = 'F:/Wetlands shapefiles/GA_shapefile_wetlands/GA_fluxdissolve.shp'

gce_fluxproper = 'F:/Wetlands shapefiles/GCE flux tower/2014-climatology.shp'

flux_ga = geemap.shp_to_ee(gce_fluxdiss) ##loop shapefile rows into gee to get around file size issue

In [4]:
Map.addLayer(flux_ga)
Map

Map(center=[31.539096, -81.422318], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [11]:
# Function to mask clouds using Sentinel-2 QA60 band
##NOTE: ONLY GOOD BEFORE JAN 2022 - see 'Georgia visualization' in ag/NAIP folder, need to use cloudless collection after 2022
def mask_clouds(image):
    # Select the QA60 band
    QA60 = image.select(['QA60'])
    # Create a cloud mask by bitwise AND-ing with 1 shifted left by 10 bits
    cloud_mask = QA60.bitwiseAnd(1 << 10).eq(0)
    # Update the image mask and return it
    return image.updateMask(cloud_mask)

In [13]:
##FLATS masking NOTE: original flats:

def addSENTINELFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B8'),
            'RED': image.select('B4'),
            'GREEN': image.select('B3')
        })
    
    return image.addBands(flats.rename('flats'))

##MASKING FLATS
def maskSENTINELFLATS(image):
    mask1 = image.select('flats').lte(0.2)
    return image.updateMask(mask1)

In [14]:
##sentinel 2 surface reflectance images - GA LTER

gaCollectionS2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(flux_ga) \
    .filter(ee.Filter.calendarRange(2017, 2019,'year')) \
    .filter(ee.Filter.calendarRange(8, 10,'month')) \
    .map(mask_clouds).map(addSENTINELFLATS).map(maskSENTINELFLATS)

##TO MASK FLOODED PIXELS, ADD:
#.map(maskSENTINELFLATS)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B8', 'B4', 'B3'], 
             'gamma': 1.5}


In [8]:
# list_s2 = gaCollectionS2.toList(gaCollectionS2.size())

# x = ee.Image(list_s2.get(0))

# Map.addLayer(x, vis_param, 'S2')
# Map.addLayer(flux_ga)

# Map

In [15]:
# list_s2.getInfo()
gaCollectionS2

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [142]:
##Pixel extraction functions - addDate for dateless images/collections
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())


In [143]:
##Adding DEM
dem = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m.
dem1 = ee.ImageCollection('USGS/3DEP/1m')

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, dem_params, '10m DEM')
# Map.addLayer(dem1, dem_params, '1m DEM')

In [144]:
#Elevation
# dem_vals = geemap.ee_to_pandas(demExtraction(dem)) ##10m dataset

In [145]:
# raster_vals = geemap.ee_to_pandas(gaCollectionS2.map(rasterExtraction))

In [157]:
## Extracting sentinel pixel values
collection = gaCollectionS2

df = pd.DataFrame(collection.aggregate_array('system:id').getInfo(),columns = ['Image'], index = None)

#print(df)

calibr = []
for i in range(len(df)):
    image = ee.Image(addSENTINELFLATS(mask_clouds(addDate(ee.Image(df.loc[i, 'Image'])))))
    try:
        region = flux_ga
        samplex = geemap.ee_to_pandas(image.addBands(image.pixelLonLat()).sampleRegions(collection = region, scale = 10, geometries = True))
        calibr.append(samplex) 
    except Exception:
        continue #Unless this exception is included, images with no pixels in shapefile (due to clouds) will result in error

image_list = pd.concat(calibr)

print(image_list)

                        WETLAND_TY  TCI_B   latitude  QA10   B8A   WVP  \
0     Estuarine and Marine Wetland    139  31.414295     0  2695  2956   
1     Estuarine and Marine Wetland    167  31.414295     0  2695  2956   
2     Estuarine and Marine Wetland    176  31.414296     0  3008  2956   
3     Estuarine and Marine Wetland    182  31.414296     0  3008  2956   
4     Estuarine and Marine Wetland    170  31.414296     0  3315  2956   
...                            ...    ...        ...   ...   ...   ...   
4053  Estuarine and Marine Wetland    126  31.405626     0  1452  2956   
4054  Estuarine and Marine Wetland    137  31.405627     0  1399  2956   
4055  Estuarine and Marine Wetland    124  31.405536     0  1452  2956   
4056  Estuarine and Marine Wetland    135  31.405536     0  1452  2956   
4057  Estuarine and Marine Wetland    142  31.405536     0  1399  2956   

         flats  longitude   B11   B12  ...    B4  TCI_R    B5    B6    B7  \
0     0.002538 -81.293684  2106  1

In [158]:

##Export as csv
in_fc = image_list
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'sentinel_soil.csv')
# image_list.to_csv(out_csv, index = False)

In [148]:
for cols in x_list.columns:
    print(cols)

percent
TCI_B
latitude
QA10
B8A
WVP
flats
longitude
B11
B12
AOT
imagedate
QA20
MSK_CLDPRB
B1
QA60
B2
B3
B4
TCI_R
B5
B6
B7
B8
B9
MSK_SNWPRB
SCL
TCI_G


In [1]:
#DEM
region = flux_ga
dem_df = geemap.ee_to_pandas(dem.addBands(dem.pixelLonLat()).sampleRegions(collection = region, scale = 10, geometries = True))

print(dem_df)

##Export as csv
in_fc = dem_df
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'DEM_soil.csv')
# dem_df.to_csv(out_csv, index = False)

NameError: name 'flux_ga' is not defined